# RNN
* Using 'neitzsche" text,I predict the upcoming letter using RNN
* I am using **theano** backend, as it is better for RNN's. 
* If **tensorflow** is used, it unrolls the whole RNN architecture and computes, whereas theano doesnt. 
  This saves a lot of memory and processing consumption. 

# Preprocess the data

In [27]:
from keras.utils.data_utils import get_file

In [28]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


Let's unroll text and make it a list of sorted chars

In [29]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars =',vocab_size)

total chars = 86


Insert '0' character for padding. 

In [30]:
chars.insert(0, "\0")

    Map chars to indices and back indices to chars

In [31]:
chars_indices = dict( (c,i) for i,c in enumerate(chars) )
indices_chars = dict( (i,c) for i,c in enumerate(chars) )

convert all the chars in text to indexes. I will be using indices data from now on

In [32]:
idx = [chars_indices[c] for c in text]

# 3 word model 
    As of now, I predict the 4th letter using previous 3 letters.

In [33]:
cs = 3
c1_dat = [idx[i] for i in range(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-1-cs, cs)]

### our inputs

In [34]:
import numpy as np

In [35]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

### Our output

In [36]:
y = np.stack(c4_dat[:-2])

### Let's create input layer and embedding layer

In [37]:
n_factors = 50

In [38]:
from keras.layers import Input, Embedding, Flatten

In [39]:
c1_in = Input(shape=(1,), dtype='int64',name='c1')
c1 = Embedding(vocab_size, n_factors, input_length=1)(c1_in)
c1 = Flatten()(c1)

c2_in = Input(shape=(1,), dtype='int64', name='c2')
c2 = Embedding(vocab_size, n_factors, input_length=1)(c2_in)
c2 = Flatten()(c2)

c3_in = Input(shape=(1,), dtype='int64', name='c3')
c3 = Embedding(vocab_size, n_factors, input_length=1)(c3_in)
c3 = Flatten()(c3)

# creating RNN without using keras.

In [40]:
from keras.layers import Dense, merge

In [41]:
#Input layer for char1
c1_in_layer = Dense(256, activation='relu')(c1)

#Input layer for char2
c2_in_layer = Dense(256, activation='relu')(c2)

#Hidden layer between c1 input to the next activation function
hidden_2 = Dense(256, activation='tanh')(c1_in_layer)

#dense of c1 and input of c2
c2_c1 = merge([c2_in_layer,hidden_2])

#Input layer for char3
c3_in_layer = Dense(256, activation='relu')(c3)

#Hidden layer between c2_out to the next activation function
hidden_3 = Dense(256, activation='tanh')(c2_c1)

#dense of c1&c2 and input of c3
c3_c2 = merge([c3_in_layer,hidden_3])

#Final output layer
c4_out = Dense(vocab_size,activation='softmax')(c3_c2)

C:\Users\Chaitra\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Chaitra\Anaconda3\lib\site-packages\keras\legacy\layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Users\Chaitra\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


### Using functional API, instead of sequential model

In [42]:
from keras.models import Model
from keras import losses 

In [43]:
rnn =  Model([c1_in,c2_in,c3_in], c4_out)

In [44]:
rnn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c1 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c2 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1, 50)         4300        c1[0][0]                         
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 1, 50)         4300        c2[0][0]                         
___________________________________________________________________________________________

In [53]:
rnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [33]:
rnn.optimizer.lr = 0.001
rnn.fit([x1,x2,x3], y, 
       epochs=8,
       batch_size=32)

Epoch 1/8
200297/200297 [==============================] - 46s - loss: 3.5877    
Epoch 2/8
200297/200297 [==============================] - 46s - loss: 3.1926    
Epoch 3/8
200297/200297 [==============================] - 46s - loss: 3.1220    
Epoch 4/8
200297/200297 [==============================] - 46s - loss: 3.0931    
Epoch 5/8
200297/200297 [==============================] - 46s - loss: 3.0733    
Epoch 6/8
200297/200297 [==============================] - 46s - loss: 3.0568    
Epoch 7/8
200297/200297 [==============================] - 46s - loss: 3.0412    
Epoch 8/8
200297/200297 [==============================] - 46s - loss: 3.0253    


In [34]:
rnn.optimizer.lr = 0.01
rnn.fit([x1,x2,x3], y, 
       epochs=8,
       batch_size=32)

Epoch 1/8
200297/200297 [==============================] - 46s - loss: 3.0088    
Epoch 2/8
200297/200297 [==============================] - 46s - loss: 2.9913    
Epoch 3/8
200297/200297 [==============================] - 47s - loss: 2.9730    
Epoch 4/8
200297/200297 [==============================] - 49s - loss: 2.9540    
Epoch 5/8
200297/200297 [==============================] - 50s - loss: 2.9346    
Epoch 6/8
200297/200297 [==============================] - 46s - loss: 2.9152    
Epoch 7/8
200297/200297 [==============================] - 48s - loss: 2.8959    
Epoch 8/8
200297/200297 [==============================] - 48s - loss: 2.8769    


# Test model

In [47]:
def next_inp(inp):
    idxs = [chars_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = rnn.predict(arrs)
    i = np.argmax(p)
    pd = chars[i]
    return pd    

In [53]:
next_inp('phi')

' '

Some where something went wrong. I will rectify the error soon.